In [13]:
import ee
import geemap
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm.notebook import tqdm
from pyproj import CRS
from shapely.geometry import Polygon, Point


sys.path.append('../../')

from src.ee_scripts.ee_utils import *

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
# =============Feature Imports================
denmark_pts = ee.FeatureCollection("projects/ee-ayang115t/assets/Marker_2018")
geometry = ee.Geometry.Polygon([[
    [9.099959167786013, 55.421683620075065],
    [9.099959167786013, 55.337420105686874],
    [9.237288269348513, 55.337420105686874],
    [9.237288269348513, 55.421683620075065]
    ]])

bounds = Polygon([
    [9.099959167786013, 55.421683620075065],
    [9.099959167786013, 55.337420105686874],
    [9.237288269348513, 55.337420105686874],
    [9.237288269348513, 55.421683620075065]
    ]
    )

# =============Extraction Variables================
random_seed = 1
start_date = '2018-01-01'
end_date = '2019-01-01'
step = 10 # date range for collected averages
pixel_limit = 1 # number of pixels to sample per feature
scale = 100 # resolution (meters) to sample in
dataset_properties = ['Afgkode', 'Afgroede'] # dataset labels to copy to SAR samples-

aoi = denmark_pts.filterBounds(geometry) #subset to investigate

In [4]:
gdf = gpd.read_file('/scratch/bbug/ayang1/raw_data/denmark_eurocrop/2018/shapefile/Marker_2018.shp')
print(len(gdf))

594433


In [5]:
translations = pd.read_csv('/scratch/bbug/ayang1/raw_data/denmark_eurocrop/2019/eurocrop_2019_translations.csv')

In [14]:
# Translate codes to HCAT3
hcat3_codes = []
hcat3_labels = []
unmatched = []
pbar = tqdm(total=len(gdf))
for val in gdf['Afgkode']:
    values = translations.loc[translations['original_code']==val]
    if len(values) == 0:
        hcat3_codes.append(np.nan)
        hcat3_labels.append(np.nan)
        unmatched.append(val)
        continue
    hcat3_codes.append(values['HCAT3_code'].values[0])
    hcat3_labels.append(values['HCAT3_name'].values[0])
    pbar.update(1)

  0%|          | 0/594433 [00:00<?, ?it/s]

 16%|█▌        | 95262/594433 [00:39<03:28, 2396.51it/s]


In [48]:
# Add HCAT3 codes and labels to gdf 
# Drop classes with less than 1000 samples, and drop untranslated values
gdf['HCAT3_code'] = hcat3_codes
gdf['HCAT3_label'] = hcat3_labels

np.random.seed(10)
data_subset = gdf.iloc[np.random.randint(0, len(gdf), 100000)]
values, counts = np.unique(data_subset['HCAT3_code'], return_counts=True)
to_drop = [values[i] for i in range(len(values)) if counts[i] < 1000]
to_drop.append(np.nan)
crop_data = data_subset.loc[~data_subset['HCAT3_code'].isin(to_drop)]
crop_data.reset_index(drop=True, inplace=True)

print('Dropped classes:', to_drop)

Dropped classes: [3301010102.0, 3301010300.0, 3301010302.0, 3301010600.0, 3301010801.0, 3301010802.0, 3301011001.0, 3301011002.0, 3301011500.0, 3301011501.0, 3301011502.0, 3301020000.0, 3301020100.0, 3301020600.0, 3301020700.0, 3301060402.0, 3301060500.0, 3301060702.0, 3301060800.0, 3301061000.0, 3301061200.0, 3301061203.0, 3301061214.0, 3301061227.0, 3301061237.0, 3301069900.0, 3301070000.0, 3301080000.0, 3301080900.0, 3301090100.0, 3301090200.0, 3301090202.0, 3301090205.0, 3301090207.0, 3301090209.0, 3301090301.0, 3301090303.0, 3301090305.0, 3301100000.0, 3301130000.0, 3301140100.0, 3301140400.0, 3301140600.0, 3301150200.0, 3301150300.0, 3301180000.0, 3301200000.0, 3301210100.0, 3301210200.0, 3301210202.0, 3301210203.0, 3301210204.0, 3301210205.0, 3301210208.0, 3301210210.0, 3301210211.0, 3301210212.0, 3301210300.0, 3301210500.0, 3301220100.0, 3301220300.0, 3301220400.0, 3301230000.0, 3301250100.0, 3301250200.0, 3301280000.0, 3301290000.0, 3301290200.0, 3301290300.0, 3301290500.0, 33

In [52]:
sampled_points = []
pbar = tqdm(total=len(crop_data))
for index, row in crop_data.iterrows():
    geometry = row['geometry']
    minx, miny, maxx, maxy = geometry.bounds
    while True:
        pnt = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
        if geometry.contains(pnt):
            new_row = row[['HCAT3_code', 'HCAT3_label']]
            new_row['geometry'] = pnt
            sampled_points.append(new_row)
            break
    pbar.update(1)

  0%|          | 0/90830 [00:00<?, ?it/s]

In [55]:
points = gpd.GeoDataFrame(sampled_points, crs=gdf.crs)
len(points)

90830

In [58]:
points.crs

<Projected CRS: EPSG:25832>
Name: ETRS89 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 6°E and 12°E: Austria; Belgium; Denmark - onshore and offshore; Germany - onshore and offshore; Norway including - onshore and offshore; Spain - offshore.
- bounds: (6.0, 38.76, 12.01, 84.33)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [56]:
points.to_file('random_subset_points.shp')

/tmp/ipykernel_1223360/3050550373.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  points.to_file('random_subset_points.shp')


In [59]:
len(pd.read_csv('/scratch/bbug/ayang1/raw_data/lucas/s1_lucas_2018/S1_point_10days_10m_1Jan-31Dec_EU_NW1_ratio-db.csv'))

172703